## 1.2 从任意长度的可迭代对象中分解元素

### 去掉首尾成绩 (快速得到中间元素) 利用  **\*表达式**，分解未知或任意长度的可迭代对象

In [1]:
from numpy import mean

In [2]:
st_scores =[1,2,3,4,5,6]
# a,b... = st_scores 这种形式或者循环或得值太麻烦

In [5]:
def drop_first_last(grades):
    firs, *middle, last =grades # *var 代表中间中间变量的列表
    return mean(middle)
drop_first_last(st_scores)

3.5

In [2]:
#分解后丢弃某些不需要的值, *_ 表示带丢弃的变量名
record =('ACM', 50 ,1234.45, (12, 18, 2012))
name, *_, (*_, year) = record
print(name)
year

ACM


2012

## 1.3 保存最后N个元素

##### 保存有限的历史记录算是collections.[deque](https://blog.csdn.net/qq_39478403/article/details/105828125)的完美应用场景 ; (类似 list 的容器，两端都能实现快速 append 和 pop (双端队列))

In [3]:
from collections import deque

In [7]:
q = deque(maxlen=2) #deque(maxlen=N)创建了一个固定长度的双向队列。当有新记录加入而队列已满时会自动移除最老的记录
q.append(1) ; q.append(2) ;q.append(3) 
q

deque([2, 3])

在队列两端执行添加和弹出,复杂度O(1);列表的头部插入或移除复杂度O(N)

In [9]:
q.appendleft(1)

In [10]:
q.popleft() #pop为右侧弹出

1

#### 对文本行做简单的文本匹配操作，当发现有匹配时就输出当前的匹配行以及最后检查过的N行文本
 注：包含[yield](https://blog.csdn.net/mieleizhi0522/article/details/82142856/)关键字，就变成了生成器函数，返回的是一个迭代器 (类似return) 

In [11]:
def search(lines,pattern,history=5):
    previous_lines = deque (maxlen=history)
    for line in lines: 
        if pattern in line: #当前行中存在匹配字符串pattern
            yield line, previous_lines
        previous_lines.append(line)  #当前行

In [22]:
def search_test():
    with open ('pline.txt')as f: 
        for line,prevlines in search(f,'python',5): #得到search函数返回的迭代器
            for pline in prevlines:
                print(pline, end='')
            print(line, end='') #默认换行，end修改
            print('**'* 4)

In [23]:
search_test()

this is test
this is test
this is test
this is python test 
********
this is test
this is test
this is test
this is python test 
this is python test 
********


## 1.4 找到最大或最小的N个元素
#### **nlargest** **nsmallest** 
n largest

In [32]:
import heapq  #堆队列算法

In [44]:
nums1=[1,8,2,23,7,-4,18,23,42,37,2]
print(heapq.nlargest(3,nums1))  #加key参数可操作更复杂的数据结构如字典等。。。
print(heapq.nsmallest(3,nums1))

[42, 37, 23]
[-4, 1, 2]


#### 如果正在寻找最大或最小的N个元素，且同集合中元素的总数目相比，N很小
`heapq.heapify(list)` 将元素以堆的顺序排列,堆最重要的特性就是list[0]总是最小那个的元素。
此外，接下来的元素可依次通过`heapq.heappop()`方法轻松找到。该方法会将第一个元素（最小的）弹出，**然后以第二小的元素取而代之**（这个操作的复杂度是O(1ogN),N代表堆的大小）

注：**列表赋值，也就是浅拷贝**，属于引用类型，被赋值的列表与源列表共同引用一个地址  
新列表的值发生改变，源列表的值也跟着变，若list_heap1 = nums1 ，则lh变得时候，nu也会改变

In [45]:
list_heap1 = list(nums1)  
heapq.heapify(list_heap1)
list_heap1 # 列表堆(小)形式满足 a[k] <= a[2*k+1] and a[k] <= a[2*k+2]

[-4, 2, 1, 23, 7, 2, 18, 23, 42, 37, 8]

In [41]:
heapq.heappop(list_heap1) 

-4

当所要找的元素数量相对较小时，函数nlargest(0和nsmallestO)才是最适用的  
如果N和集合本身的大小差不多大，通常更快的方法是先对集合排序，然后做切片操作

## 1.5 实现优先级队列

#### 我们想要实现一个队列，它能够以给定的优先级来对元素排序，且每次pop操作时都会返回优先级最高的那个元素。

In [59]:
class PriorityQueue: 
    def __init__(self) -> None:
        self._queue = []  #保证属性不重复，可以在其前面加上单个下划线
        self._index = 0

    def push(self, item, priority):
        heapq.heappush(self._queue, (-priority, self._index, item))  #将一个元组(-priority,index,item)作为整体添加到队列。
        self._index += 1  #priority相同时比较index
    
    def pop(self):
        return heapq.heappop(self.queue)[-1] #返回元素为  按-priority和index排序后第一个元素的最后一个元素

In [57]:
#演示如何使用上述类
class Item: 
    def __init__ (self, name):
         self.name = name 
    def __repr__ (self): 
        return 'Item ({!r})'.format (self.name)

In [58]:
pq = PriorityQueue()
pq.push(Item('foo'), 1)

上面的代码片段的核心在于heapq模块的使用。函数heapq.heappush()以及heapq.heappop(分别实现将元素从列表queue中插入和移除，且保证列表中第一个元素的优先级最低（如1.4节所述)。  
heappop(0方法总是返回“最小”的元素，因此这就是让队列能弹出正确元素的关键。此外，由于push和pop操作的复杂度都是O(logN), 其中N代表堆中元素的数量，因此就算N的值很大，这些操作的效率也非常高。  
把priority取负值是为了让队列能够按元素的优先级从高到低的顺序排列。  这和正常的堆排列顺序相反，一般情况下堆是按从小到大的顺序排序的。
变量index的作用是为了将具有相同优先级的元素以适当的顺序排列。  
通过维护一个不断递增的索引，元素将以它们入队列时的顺序来排列。但是，index在对具有相同优先级的元素间做比较操作时同样扮演了重要的角色。